# Notes
Here are notes, or code that was not needed in the final analysis, but could be handy.

In [1]:
import pandas as pd
from sspipe import p, px
import re

In [2]:
# read files
df_prod_tech_specs = pd.read_csv("data/products_technical_specs.csv")
df_prod_titles = pd.read_csv("data/products_titles.csv")

In [3]:
# parse specs and create columns for every spec (this was not needed in the end)

def check_for_two_elements(arr):
    """ Check for a length of exactly 2 """
    if len(arr) != 2:
        raise Exception("Data in wrong format", arr)
    return arr


# Im using pipe notation here, which is enabled by sspipe (more info here: https://github.com/sspipe/sspipe)
df_new_spec_columns = (
        df_prod_tech_specs["Product_tech_specs"]
        | p(map, (
            p(re.sub, r"\|+", "|", px)  # remove repeating and trailing delimiters
            | px.strip("|")
            | p(re.sub, r"\|Löt", ", Löt", px)  # Fix one product that uses | in a spec
            | px.split("|")  # split into single specs (e.g. "AWG: 18")
            | p(map, (
                px.split(':', 1)  # split single specs into name, value pairs (e.g. ["AWG", "18"])
                | p(map, px.strip(" ,"))  # trim trailing spaces and comma
                | p(list)
                | p(check_for_two_elements)  # check, that there are exactly two elements (name, value)
            ))
            | p(pd.DataFrame)
            | px.set_index(0)
            | px.transpose()
        ))
        | p(pd.concat)
        | px.reset_index(drop=True)
)

print("New columns shape: ", df_new_spec_columns.shape)
# df_new_spec_columns

df_prod_tech_specs_parsed = df_prod_tech_specs.merge(df_new_spec_columns, left_index=True, right_index=True)
df_prod_tech_specs_parsed

New columns shape:  (26, 82)


,Product_No,Product_tech_specs,AWG,Abschirmung,Anschluss-Typ,Anschlusstyp A (bzw. Eingänge),Anschlusstyp B (bzw. Ausgänge),Anzahl Anschluss A,Anzahl Anschluss B,Herstellerfarbe,...,"Produktabmessung, Tiefe",Ausführung (Gewindebohrer),Drehrichtung Gewinde,Gewinde-Länge,Gewinde-Maß,Gewindeart,Kernloch-Größe,Schaftvierkant,Steigung,Anschnittform
0,1000,AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...,18,ohne Schirmung,IDE,IDE-Strom-Stecker 4pol.,IDE-Strom-Buchse 4pol.,1 x,2 x,"Schwarz, Rot, Gelb",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...,18,ohne Schirmung,Mainboard,ATX-Strom-Stecker 14pol.,ATX-Strom-Buchse 24pol.,1 x,1 x,Bunt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1002,"Anschluss-Typ: SATA, PCI-Express|Anschlusstyp ...",NaN,NaN,"SATA, PCI-Express",SATA-Strom-Stecker 15pol.,PCIe-Stecker 6pol.,2 x,1 x,"Schwarz, Gelb",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,Anschluss-Typ: Kaltgeräte|Anschlusstyp A (bzw....,NaN,NaN,Kaltgeräte,Schutzkontakt-Stecker,Kaltgeräte-Buchse C13,1 x,1 x,Schwarz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1004,AWG: 26/7|Anschluss-Typ: RJ45|Anschlusstyp A (...,26/7,NaN,RJ45,RJ45-Stecker,RJ45-Stecker,1 x,1 x,Gelb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1005,Anschlusstyp A (bzw. Eingänge): Kaltgeräte-Ste...,NaN,NaN,NaN,Kaltgeräte-Stecker C14,"Kaltgeräte-Buchse C13, Kaltgeräte-Buchse C13",NaN,NaN,Schwarz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1007,Anschlusstyp A (bzw. Eingänge): Kleeblatt-...,NaN,NaN,NaN,NaN,offene Kabelenden,NaN,NaN,Schwarz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1008,Anschlusstyp A (bzw. Eingänge): PowerCon-Buchs...,NaN,NaN,NaN,PowerCon-Buchse,offene Kabelenden,NaN,NaN,Schwarz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1009,Anschlusstyp A (bzw. Eingänge): Schutzkontakt-...,NaN,NaN,NaN,Schutzkontakt-Gummi-Stecker,Schutzkontakt-Gummi-Kupplung,NaN,NaN,Rot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1006,AWG: 26/7|Anschluss-Typ: RJ45|||Anschlusstyp A...,26/7,NaN,RJ45,RJ45-Stecker,RJ45-Buchse,1 x,1 x,Violett,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
